In [21]:
### Helper Functions and global settings
FORCE_CPU = False
line_length = 60
def pretty_print_divider(n=1, lb_n=0, char="#"):
    if isinstance(n, bool):
        n = 1 if n else 0
    if lb_n > 0:
        print("\n" * lb_n, end="")
    elif n > 1:
        print()
    for _ in range(n):
        print(char * line_length)

def pretty_print(message, pb=False, pa=False, lb_n=0, char="#"):
    pretty_print_divider(pb, lb_n=lb_n, char=char)
    available_space = line_length - 7
    formatted_message = f"{char * 3} {message}"
    padding = line_length - len(formatted_message) - 4
    if padding < 0:
        formatted_message = formatted_message[:line_length-7] + "..."
        padding = 0
    print(formatted_message + " " * padding + f" {char * 3}")
    pretty_print_divider(pa, char=char)

In [22]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

torch.manual_seed(8) # for reproduce
# Dynamically set the device to use (either GPU or CPU)
if torch.cuda.is_available() and not FORCE_CPU:
    device_count = torch.cuda.device_count()
    if device_count > 1:
        pretty_print(f"multiple GPUs detected ({device_count})", pb=True)
        best_device_id = -1
        max_free_mem = 0
        for i in range(device_count):
            free, _ = torch.cuda.mem_get_info(i)
            pretty_print(f"GPU {i} free memory: {free}")
            if free > max_free_mem:
                max_free_mem = free
                best_device_id = i
        device = torch.device(f'cuda:{best_device_id}')
        out_device = f"GPU: cuda:{best_device_id}"
    else:
        device = torch.device('cuda')
        out_device = "GPU: cuda:0"
else:
    device = torch.device('cpu')
    out_device = "CPU"
torch.backends.cudnn.benchmark = True
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)
torch.nn.Module.dump_patches = True

pretty_print(f"Using device: {out_device}", pb=True, pa=True)

############################################################
### multiple GPUs detected (2)                           ###
### GPU 0 free memory: 3836477440                        ###
### GPU 1 free memory: 5770051584                        ###
############################################################
### Using device: GPU: cuda:1                            ###
############################################################


In [23]:
import time
import numpy as np
import gc
import sys
sys.setrecursionlimit(50000)
import pickle

# from tensorboardX import SummaryWriter

import copy
import pandas as pd
#then import my own modules
from AttentiveFP import Fingerprint, Fingerprint_viz, save_smiles_dicts, get_smiles_dicts, get_smiles_array, moltosvg_highlight

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import f1_score

In [25]:
# from rdkit.Chem import rdMolDescriptors, MolSurf
# from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
# from rdkit.Chem import AllChem
from rdkit.Chem import QED
# get_ipython().run_line_magic('matplotlib', 'inline')
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from IPython.display import SVG, display
import seaborn as sns; sns.set(color_codes=True)

In [26]:
task_name = 'BBBP'
tasks = ['BBBP']
raw_filename = "../data/BBBP.csv"
feature_filename = raw_filename.replace('.csv','.pickle')
filename = raw_filename.replace('.csv','')
prefix_filename = raw_filename.split('/')[-1].replace('.csv','')
smiles_tasks_df = pd.read_csv(raw_filename)
smilesList = smiles_tasks_df.smiles.values
pretty_print(f"number of all smiles: {len(smilesList)}")
atom_num_dist = []
remained_smiles = []
canonical_smiles_list = []
for smiles in smilesList:
    try:        
        mol = Chem.MolFromSmiles(smiles)
        atom_num_dist.append(len(mol.GetAtoms()))
        remained_smiles.append(smiles)
        canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
    except:
        pretty_print(f"not successfully processed smiles: {smiles}")
        pass
pretty_print(f"number of successfully processed smiles: {len(remained_smiles)}")
smiles_tasks_df = smiles_tasks_df[smiles_tasks_df["smiles"].isin(remained_smiles)]
# print(smiles_tasks_df)
smiles_tasks_df['cano_smiles'] =canonical_smiles_list
assert canonical_smiles_list[8]==Chem.MolToSmiles(Chem.MolFromSmiles(smiles_tasks_df['cano_smiles'][8]), isomericSmiles=True)

plt.figure(figsize=(5, 3))
sns.set(font_scale=1.5)
ax = sns.distplot(atom_num_dist, bins=28, kde=False)
plt.tight_layout()
# plt.savefig("atom_num_dist_"+prefix_filename+".png",dpi=200)
plt.show()
plt.close()

# print(len([i for i in atom_num_dist if i<51]),len([i for i in atom_num_dist if i>50]))

### number of all smiles: 2050                           ###
### not successfully processed smiles: O=N([O-])C1=C(... ###
### not successfully processed smiles: c1(nc(NC(N)=[N... ###
### not successfully processed smiles: Cc1nc(sc1)\[NH... ###


[00:15:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] Explicit valence for atom # 6 N, 4, is greater than permitted
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57] WARNING: not removing hydrogen atom without neighbors
[00:15:57

### not successfully processed smiles: s1cc(CSCCN\C(N... ###
### not successfully processed smiles: c1c(c(ncc1)CSC... ###
### not successfully processed smiles: n1c(csc1\[NH]=... ###
### not successfully processed smiles: n1c(csc1\[NH]=... ###
### not successfully processed smiles: n1c(csc1\[NH]=... ###
### not successfully processed smiles: n1c(csc1\[NH]=... ###
### not successfully processed smiles: s1cc(nc1\[NH]=... ###
### not successfully processed smiles: c1(cc(N\C(=[NH... ###


[00:15:58] Explicit valence for atom # 12 N, 4, is greater than permitted
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] Explicit valence for atom # 5 N, 4, is greater than permitted
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing 

### number of successfully processed smiles: 2039        ###


[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
[00:15:58] WARNING: not removing hydrogen atom without neighbors
/tmp/ipykernel_342953/2913970869.py:30: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  ax = sns.distplot(atom_num_dist, bins=28, kde=False)
/tmp/ipykernel_342953/2913970869.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
random_seed = 188
random_seed = int(time.time())
start_time = str(time.ctime()).replace(':','-').replace(' ','_')
start = time.time()

batch_size = 100
epochs = 800
p_dropout = 0.1
fingerprint_dim = 150

radius = 3
T = 2
weight_decay = 2.9 # also known as l2_regularization_lambda
learning_rate = 3.5
per_task_output_units_num = 2 # for classification model with 2 classes
output_units_num = len(tasks) * per_task_output_units_num

In [28]:
smilesList = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms())<101]
uncovered = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms())>100]

smiles_tasks_df = smiles_tasks_df[~smiles_tasks_df["cano_smiles"].isin(uncovered)]

if os.path.isfile(feature_filename):
    feature_dicts = pickle.load(open(feature_filename, "rb" ))
else:
    feature_dicts = save_smiles_dicts(smilesList,filename)
# feature_dicts = get_smiles_dicts(smilesList)

remained_df = smiles_tasks_df[smiles_tasks_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
uncovered_df = smiles_tasks_df.drop(remained_df.index)
uncovered_df

[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not removing hydrogen atom without neighbors
[00:16:01] WARNING: not r

,BBBP,smiles,cano_smiles


In [29]:
weights = []
for i,task in enumerate(tasks):    
    negative_df = remained_df[remained_df[task] == 0][["smiles",task]]
    positive_df = remained_df[remained_df[task] == 1][["smiles",task]]
    weights.append([(positive_df.shape[0]+negative_df.shape[0])/negative_df.shape[0],\
                    (positive_df.shape[0]+negative_df.shape[0])/positive_df.shape[0]])

test_df = remained_df.sample(frac=1/10, random_state=random_seed) # test set
training_data = remained_df.drop(test_df.index) # training data

# training data is further divided into validation set and train set
valid_df = training_data.sample(frac=1/9, random_state=random_seed) # validation set
train_df = training_data.drop(valid_df.index) # train set
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

valid_df.to_csv('sets/valid_df.csv')
train_df.to_csv('sets/train_df.csv')
test_df.to_csv('sets/test_df.csv')

In [30]:
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([smilesList[0]],feature_dicts)
num_atom_features = x_atom.shape[-1]
num_bond_features = x_bonds.shape[-1]

loss_function = [nn.CrossEntropyLoss(torch.tensor(weight),reduction='mean') for weight in weights]
model = Fingerprint(radius, T, num_atom_features,num_bond_features,
            fingerprint_dim, output_units_num, p_dropout)
model.to(device)
# tensorboard = SummaryWriter(log_dir="runs/"+start_time+"_"+prefix_filename+"_"+str(fingerprint_dim)+"_"+str(p_dropout))

# optimizer = optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
optimizer = optim.Adam(model.parameters(), 10**-learning_rate, weight_decay=10**-weight_decay)
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
pretty_print(f"Number of parameters: {params}")
for name, param in model.named_parameters():
    if param.requires_grad:
        pretty_print(f"{name} {param.data.shape}")



### Number of parameters: 649206                         ###
### atom_fc.weight torch.Size([150, 39])                 ###
### atom_fc.bias torch.Size([150])                       ###
### neighbor_fc.weight torch.Size([150, 49])             ###
### neighbor_fc.bias torch.Size([150])                   ###
### GRUCell.0.weight_ih torch.Size([450, 150])           ###
### GRUCell.0.weight_hh torch.Size([450, 150])           ###
### GRUCell.0.bias_ih torch.Size([450])                  ###
### GRUCell.0.bias_hh torch.Size([450])                  ###
### GRUCell.1.weight_ih torch.Size([450, 150])           ###
### GRUCell.1.weight_hh torch.Size([450, 150])           ###
### GRUCell.1.bias_ih torch.Size([450])                  ###
### GRUCell.1.bias_hh torch.Size([450])                  ###
### GRUCell.2.weight_ih torch.Size([450, 150])           ###
### GRUCell.2.weight_hh torch.Size([450, 150])           ###
### GRUCell.2.bias_ih torch.Size([450])                  ###
### GRUCell.2.bias_hh to

In [31]:
def train(model, dataset, optimizer, loss_function):
    model.train()
    np.random.seed(epoch)
    valList = np.arange(0,dataset.shape[0])
    #shuffle them
    np.random.shuffle(valList)
    batch_list = []
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)   
    for counter, train_batch in enumerate(batch_list):
        batch_df = dataset.loc[train_batch,:]
        smiles_list = batch_df.cano_smiles.values

        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        atoms_prediction, mol_prediction = model(torch.tensor(x_atom),
                                                 torch.tensor(x_bonds),
                                                 torch.tensor(x_atom_index, dtype=torch.long),
                                                 torch.tensor(x_bond_index, dtype=torch.long),
                                                 torch.tensor(x_mask))
#         print(torch.Tensor(x_atom).size(),torch.Tensor(x_bonds).size(),torch.cuda.LongTensor(x_atom_index).size(),torch.cuda.LongTensor(x_bond_index).size(),torch.Tensor(x_mask).size())

        model.zero_grad()
        # Step 4. Compute your loss function. (Again, Torch wants the target wrapped in a variable)
        loss = 0.0
        for i,task in enumerate(tasks):
            y_pred = mol_prediction[:, i * per_task_output_units_num:(i + 1) *
                                    per_task_output_units_num]
            y_val = batch_df[task].values

            validInds = np.where((y_val==0) | (y_val==1))[0]
#             validInds = np.where(y_val != -1)[0]
            if len(validInds) == 0:
                continue
            y_val_adjust = np.array([y_val[v] for v in validInds]).astype(float)
            validInds = torch.tensor(validInds, dtype=torch.long).squeeze()
            y_pred_adjust = torch.index_select(y_pred, 0, validInds)

            loss += loss_function[i](
                y_pred_adjust,
                torch.tensor(y_val_adjust, dtype=torch.long))
        # Step 5. Do the backward pass and update the gradient
#             print(y_val,y_pred,validInds,y_val_adjust,y_pred_adjust)
        loss.backward()
        optimizer.step()
def eval(model, dataset):
    model.eval()
    y_val_list = {}
    y_pred_list = {}
    losses_list = []
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)   
    for counter, test_batch in enumerate(batch_list):
        batch_df = dataset.loc[test_batch,:]
        smiles_list = batch_df.cano_smiles.values

        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        atoms_prediction, mol_prediction = model(torch.tensor(x_atom),
                                                 torch.tensor(x_bonds),
                                                 torch.tensor(x_atom_index, dtype=torch.long),
                                                 torch.tensor(x_bond_index, dtype=torch.long),
                                                 torch.tensor(x_mask))
        atom_pred = atoms_prediction.data[:,:,1].unsqueeze(2).cpu().numpy()
        for i,task in enumerate(tasks):
            y_pred = mol_prediction[:, i * per_task_output_units_num:(i + 1) *
                                    per_task_output_units_num]
            y_val = batch_df[task].values

            validInds = np.where((y_val==0) | (y_val==1))[0]
#             validInds = np.where((y_val=='0') | (y_val=='1'))[0]
#             print(validInds)
            if len(validInds) == 0:
                continue
            y_val_adjust = np.array([y_val[v] for v in validInds]).astype(float)
            validInds = torch.tensor(validInds, dtype=torch.long).squeeze()
            y_pred_adjust = torch.index_select(y_pred, 0, validInds)
#             print(validInds)
            loss = loss_function[i](
                y_pred_adjust,
                torch.tensor(y_val_adjust, dtype=torch.long))
#             print(y_pred_adjust)
            y_pred_adjust = F.softmax(y_pred_adjust,dim=-1).data.cpu().numpy()[:,1]
            losses_list.append(loss.cpu().detach().numpy())
            try:
                y_val_list[i].extend(y_val_adjust)
                y_pred_list[i].extend(y_pred_adjust)
            except:
                y_val_list[i] = []
                y_pred_list[i] = []
                y_val_list[i].extend(y_val_adjust)
                y_pred_list[i].extend(y_pred_adjust)
#             print(y_val,y_pred,validInds,y_val_adjust,y_pred_adjust)            
    test_roc = [roc_auc_score(y_val_list[i], y_pred_list[i]) for i in range(len(tasks))]
    test_prc = [auc(precision_recall_curve(y_val_list[i], y_pred_list[i])[1],precision_recall_curve(y_val_list[i], y_pred_list[i])[0]) for i in range(len(tasks))]
#     test_prc = auc(recall, precision)
    test_precision = [precision_score(y_val_list[i],
                                     (np.array(y_pred_list[i]) > 0.5).astype(int)) for i in range(len(tasks))]
    test_recall = [recall_score(y_val_list[i],
                               (np.array(y_pred_list[i]) > 0.5).astype(int)) for i in range(len(tasks))]
    test_loss = np.array(losses_list).mean()

    return test_roc, test_prc, test_precision, test_recall, test_loss

In [11]:
best_param ={}
best_param["roc_epoch"] = 0
best_param["loss_epoch"] = 0
best_param["valid_roc"] = 0
best_param["valid_loss"] = 9e8

for epoch in range(epochs):    
    train_roc, train_prc, train_precision, train_recall, train_loss = eval(model, train_df)
    valid_roc, valid_prc, valid_precision, valid_recall, valid_loss = eval(model, valid_df)
    train_roc_mean = np.array(train_roc).mean()
    valid_roc_mean = np.array(valid_roc).mean()

#     tensorboard.add_scalars('ROC',{'train_roc':train_roc_mean,'valid_roc':valid_roc_mean},epoch)
#     tensorboard.add_scalars('Losses',{'train_losses':train_loss,'valid_losses':valid_loss},epoch)

    if valid_roc_mean > best_param["valid_roc"]:
        best_param["roc_epoch"] = epoch
        best_param["valid_roc"] = valid_roc_mean
        if valid_roc_mean > 0.87:
             torch.save(model, 'saved_models/model_'+prefix_filename+'_'+start_time+'_'+str(epoch)+'.pt')             
    if valid_loss < best_param["valid_loss"]:
        best_param["loss_epoch"] = epoch
        best_param["valid_loss"] = valid_loss

    pretty_print(f"EPOCH: {epoch}", pb=3 if epoch == 0 else False)
    pretty_print(f"train_roc: {train_roc}")
    pretty_print(f"valid_roc: {valid_roc}", pa=True)

    if (epoch - best_param["roc_epoch"] >18) and (epoch - best_param["loss_epoch"] >28):        
        break

    train(model, train_df, optimizer, loss_function)

/home/herget/conda/envs/lea/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


EPOCH:	0
train_roc:[0.6769901056304918]
valid_roc:[0.686698717948718]

EPOCH:	1
train_roc:[0.7555725047277743]
valid_roc:[0.7243589743589743]

EPOCH:	2
train_roc:[0.7906175883649738]
valid_roc:[0.747596153846154]

EPOCH:	3
train_roc:[0.8007355476382025]
valid_roc:[0.7530715811965812]

EPOCH:	4
train_roc:[0.8108576861593756]
valid_roc:[0.7564102564102564]

EPOCH:	5
train_roc:[0.8183468984756193]
valid_roc:[0.7685630341880343]

EPOCH:	6
train_roc:[0.8218783629885802]
valid_roc:[0.7620192307692307]



KeyboardInterrupt: 

In [ ]:
# evaluate model
best_model = torch.load('saved_models/model_'+prefix_filename+'_'+start_time+'_'+str(best_param["roc_epoch"])+'.pt')     

# best_model_dict = best_model.state_dict()
# best_model_wts = copy.deepcopy(best_model_dict)

# model.load_state_dict(best_model_wts)
# (best_model.align[0].weight == model.align[0].weight).all()

test_roc, test_prc, test_precision, test_recall, test_losses = eval(best_model, test_df)

pretty_print(f"best epoch: {best_param['roc_epoch']}", pb=True)
pretty_print(f"test_roc: {test_roc}")
pretty_print(f"test_roc_mean: {np.array(test_roc).mean()}", pa=True)